In [1]:
#importing anticipated modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from matplotlib.mlab import PCA as mlabPCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 
import scipy
from scipy import stats
from scipy.stats import ttest_ind
from scipy.stats import pearsonr
import datetime
import collections
import statsmodels.stats.proportion
%matplotlib inline

In [2]:
# Set the default plot aesthetics to be prettier.
sns.set_style("white")

In [3]:
#importng the target csv files
train_df = pd.read_csv('/Users/manueltanguma/Downloads/data_set_ALL_AML_train.csv', encoding='latin-1')
indy_df = pd.read_csv('/Users/manueltanguma/Downloads/data_set_ALL_AML_independent.csv', encoding='latin-1')
act_df = pd.read_csv('/Users/manueltanguma/Downloads/actual_ALL_AML.csv', encoding='latin-1')

In [4]:
train_df.shape

(7129, 78)

In [5]:
indy_df.shape

(7129, 70)

In [6]:
train_df.head(2)

,Gene Description,Gene Accession Number,1,call,2,call.1,3,call.2,4,call.3,...,29,call.33,30,call.34,31,call.35,32,call.36,33,call.37
0,AFFX-BioB-5_at (endogenous control),AFFX-BioB-5_at,-214,A,-139,A,-76,A,-135,A,...,15,A,-318,A,-32,A,-124,A,-135,A
1,AFFX-BioB-M_at (endogenous control),AFFX-BioB-M_at,-153,A,-73,A,-49,A,-114,A,...,-114,A,-192,A,-49,A,-79,A,-186,A


In [7]:
indy_df.head(2)

,Gene Description,Gene Accession Number,39,call,40,call.1,42,call.2,47,call.3,...,65,call.29,66,call.30,63,call.31,64,call.32,62,call.33
0,AFFX-BioB-5_at (endogenous control),AFFX-BioB-5_at,-342,A,-87,A,22,A,-243,A,...,-62,A,-58,A,-161,A,-48,A,-176,A
1,AFFX-BioB-M_at (endogenous control),AFFX-BioB-M_at,-200,A,-248,A,-153,A,-218,A,...,-198,A,-217,A,-215,A,-531,A,-284,A


In [8]:
#making the columns easier to work with
train_df.columns = [c.lower().replace(' ', '_') for c in train_df.columns]

In [9]:
indy_df.columns = [c.lower().replace(' ', '_') for c in indy_df.columns]

In [10]:
train_df1 = [col for col in train_df.columns if "call" not in col]
train_df = train_df[train_df1]
train_df.head(2)

,gene_description,gene_accession_number,1,2,3,4,5,6,7,8,...,35,36,37,38,28,29,30,31,32,33
0,AFFX-BioB-5_at (endogenous control),AFFX-BioB-5_at,-214,-139,-76,-135,-106,-138,-72,-413,...,7,-213,-25,-72,-4,15,-318,-32,-124,-135
1,AFFX-BioB-M_at (endogenous control),AFFX-BioB-M_at,-153,-73,-49,-114,-125,-85,-144,-260,...,-100,-252,-20,-139,-116,-114,-192,-49,-79,-186


In [11]:
indy_df1 = [col for col in indy_df.columns if "call" not in col]
indy_df = indy_df[indy_df1]
indy_df.head(2)

,gene_description,gene_accession_number,39,40,42,47,48,49,41,43,...,54,57,58,60,61,65,66,63,64,62
0,AFFX-BioB-5_at (endogenous control),AFFX-BioB-5_at,-342,-87,22,-243,-130,-256,-62,86,...,-90,-137,-157,-172,-47,-62,-58,-161,-48,-176
1,AFFX-BioB-M_at (endogenous control),AFFX-BioB-M_at,-200,-248,-153,-218,-177,-249,-23,-36,...,-87,-51,-370,-122,-442,-198,-217,-215,-531,-284


In [12]:
act_df.head(5)

,patient,cancer
0,1,ALL
1,2,ALL
2,3,ALL
3,4,ALL
4,5,ALL


In [13]:
act_df_tran = act_df.T
act_df = act_df_tran
act_df.head (5)

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
patient,1,2,3,4,5,6,7,8,9,10,...,63,64,65,66,67,68,69,70,71,72
cancer,ALL,ALL,ALL,ALL,ALL,ALL,ALL,ALL,ALL,ALL,...,AML,AML,AML,AML,ALL,ALL,ALL,ALL,ALL,ALL


In [14]:
train_df2 = train_df.drop(['gene_description','gene_accession_number'],axis=1)
#train_df2.index = pd.to_numeric(train_df2.index)
#train_df2.sort_index(inplace=True)
train_df2.head()

,1,2,3,4,5,6,7,8,9,10,...,35,36,37,38,28,29,30,31,32,33
0,-214,-139,-76,-135,-106,-138,-72,-413,5,-88,...,7,-213,-25,-72,-4,15,-318,-32,-124,-135
1,-153,-73,-49,-114,-125,-85,-144,-260,-127,-105,...,-100,-252,-20,-139,-116,-114,-192,-49,-79,-186
2,-58,-1,-307,265,-76,215,238,7,106,42,...,-57,136,124,-1,-125,2,-95,49,-37,-70
3,88,283,309,12,168,71,55,-2,268,219,...,132,318,325,392,241,193,312,230,330,337
4,-295,-264,-376,-419,-230,-272,-399,-541,-210,-178,...,-377,-209,-396,-324,-191,-51,-139,-367,-188,-407


In [15]:
print (train_df2)

          1      2      3      4      5      6      7      8      9     10  \
0      -214   -139    -76   -135   -106   -138    -72   -413      5    -88   
1      -153    -73    -49   -114   -125    -85   -144   -260   -127   -105   
2       -58     -1   -307    265    -76    215    238      7    106     42   
3        88    283    309     12    168     71     55     -2    268    219   
4      -295   -264   -376   -419   -230   -272   -399   -541   -210   -178   
5      -558   -400   -650   -585   -284   -558   -551   -790   -535   -246   
6       199   -330     33    158      4     67    131   -275      0    328   
7      -176   -168   -367   -253   -122   -186   -179   -463   -174   -148   
8       252    101    206     49     70     87    126     70     24    177   
9       206     74   -215     31    252    193    -20   -169    506    183   
10      -41     19     19    363    155    325   -115    361    284   -143   
11     -831   -743  -1135   -934   -471   -631  -1003  -1001   -

In [16]:
list(train_df2.columns.values)

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '34',
 '35',
 '36',
 '37',
 '38',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33']

In [17]:
list(act_df.columns.values)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71]